In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('../data/data.csv')
#let's remove columns we can't use
data=data[['date_time', 'zone_id', 'banner_id', 'campaign_clicks',
       'os_id', 'country_id', 'impressions', 'clicks']]
data.head()

,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,0,1,1
1,2021-09-26 22:54:49.000000,1,1,0,0,1,1,1
2,2021-09-26 23:57:20.000000,2,2,3,0,0,1,1
3,2021-09-27 00:04:30.000000,3,3,0,1,1,1,1
4,2021-09-27 00:06:21.000000,4,4,0,1,0,1,1


In [3]:
data.columns

Index(['date_time', 'zone_id', 'banner_id', 'campaign_clicks', 'os_id',
       'country_id', 'impressions', 'clicks'],
      dtype='object')

Посмотрим на колонки, которые будем использовать

In [4]:
data['os_id'].value_counts()

2     4589979
0     3856798
1     3178693
4     3012541
3      759767
6      310346
5      111946
8        1273
7          99
9          25
10          5
Name: os_id, dtype: int64

Здесь мы видим 7 (от 0 до 6) главных операционных систем, и 7 маленьких.

Есть ли у них корреляция с таргетом?

In [5]:
pd.crosstab(data['os_id'],data['clicks'], normalize='index')

clicks,0,1
os_id,,
0,0.969938,0.030062
1,0.986482,0.013518
2,0.982168,0.017832
3,0.951956,0.048044
4,0.960417,0.039583
5,0.946698,0.053302
6,0.936397,0.063603
7,0.969697,0.030303
8,0.975648,0.024352


Разница есть, так что сделаем one-hot encoding для основных операционок (их 7)

Теперь посмотрим на страну

In [6]:
data['country_id'].value_counts()

0     4956393
5     1910054
6     1582705
7     1525569
1     1346321
3     1337392
15     603301
12     478038
9      439004
4      388801
10     356490
11     212245
8      210211
13     190855
14     138385
16      79166
2       66542
Name: country_id, dtype: int64

Для всех стран достаточно данных, убирать никакие не будем

In [7]:
pd.crosstab(data['country_id'],data['clicks'], normalize='index')

clicks,0,1
country_id,,
0,0.972633,0.027367
1,0.964725,0.035275
2,0.966532,0.033468
3,0.972277,0.027723
4,0.974380,0.025620
5,0.977401,0.022599
6,0.983186,0.016814
7,0.973203,0.026797
8,0.955763,0.044237


Страна тоже имеет смысл. Если бы были названия стран, то, скорее всего, можно было бы некоторые собрать вместе (Европу, или какую-то ее часть, например), но сейчас мы это сделать не можем.

С зоной все довольно странно - нет какой-то "условно-линейной" зависимости: у зоны 4 конверсия больше, чем у зоны 3, например. У зоны 8 внезапно большая... Поэтому я сделаю фичи для зон 0,1,2,3,4,5-9, 10-20, 20-50, >50

In [8]:
pd.crosstab(data[data['zone_id']<=20]['zone_id'], data[data['zone_id']<=20]['clicks'], normalize='index')

clicks,0,1
zone_id,,
0,0.992747,0.007253
1,0.959501,0.040499
2,0.974926,0.025074
3,0.974469,0.025531
4,0.964719,0.035281
5,0.982495,0.017505
6,0.963986,0.036014
7,0.977471,0.022529
8,0.943294,0.056706


Теперь посмотрим на дату и время

In [9]:
data['date'] = pd.to_datetime(data['date_time']).dt.date

In [10]:
data['date'].value_counts()

2021-09-26    3102610
2021-09-29    2420588
2021-09-27    2367303
2021-09-28    2307355
2021-10-02    2128978
2021-09-30    1851189
2021-10-01    1643448
2021-09-01          1
Name: date, dtype: int64

In [11]:
data['hour'] = pd.to_datetime(data['date_time']).dt.hour

In [12]:
data['hour'].value_counts()

19    1135143
20    1082943
18    1014305
16     918587
17     910469
21     812888
15     744556
14     716351
22     667214
13     634799
12     625483
11     592749
23     578167
0      550788
8      547525
9      516624
10     509720
1      498533
2      482018
7      481859
6      476654
5      474251
3      444033
4      405813
Name: hour, dtype: int64

In [13]:
pd.crosstab(data['hour'],data['clicks'], normalize='index')

clicks,0,1
hour,,
0,0.972873,0.027127
1,0.974166,0.025834
2,0.973887,0.026113
3,0.973479,0.026521
4,0.974959,0.025041
5,0.975138,0.024862
6,0.974667,0.025333
7,0.974710,0.025290
8,0.976533,0.023467


Разницы практически нет, что странно. Возможно, это работает вместе со страной (если есть тайм зоны)? 

In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [15]:
data.groupby(['hour', 'country_id']).clicks.value_counts(normalize=True)

hour  country_id  clicks
0     0           0         0.974442
                  1         0.025558
      1           0         0.965180
                  1         0.034820
      2           0         0.961574
                  1         0.038426
      3           0         0.972732
                  1         0.027268
      4           0         0.970271
                  1         0.029729
      5           0         0.977706
                  1         0.022294
      6           0         0.982807
                  1         0.017193
      7           0         0.971735
                  1         0.028265
      8           0         0.963632
                  1         0.036368
      9           0         0.963736
                  1         0.036264
      10          0         0.984748
                  1         0.015252
      11          0         0.967649
                  1         0.032351
      12          0         0.978493
                  1         0.021507
      13     

In [16]:
data.groupby(['hour', 'country_id']).size()

hour  country_id
0     0             287343
      1              97932
      2               1093
      3              28018
      4               5651
      5              37589
      6              21229
      7              22466
      8               4042
      9               7997
      10              7212
      11              5873
      12              6556
      13              2812
      14              3278
      15             10675
      16              1022
1     0             292352
      1             101522
      2                793
      3              18158
      4               3280
      5              25825
      6              14361
      7              11973
      8               2655
      9               4670
      10              4906
      11              2690
      12              3785
      13              1904
      14              1996
      15              6893
      16               770
2     0             307974
      1              98734
      2    

Вообще говоря, разница есть, но есть подозрение, что его оверфитнет. Стоит ли добавлять такое декартово произведение - хороший вопрос. По логике, было бы неплохо для каждой страны выделить утро, день и вечер, но у нас нет названий. Я лучше сделаю декартово произведение страны с баннером

upd Даже модель без декартова произведения работала довольно долго, поэтому интеракцию страны с баннером я решила не делать...

Это, кажется, ненужная колонка

In [17]:
data['impressions'].value_counts()

1    15821472
Name: impressions, dtype: int64

А сколько баннеров?

In [18]:
len(data['banner_id'].unique())

1633

Кажется, что, если показов было меньше сотни, то мы нас оверфитнет. Поэтому, возможно, стоит запихнуть все эти баннера (про которые мы мало что знаем) в один тип (-1)

In [19]:
data['banner_id'].value_counts()

22      613367
361     387563
3       286999
18      262946
21      246378
52      227213
16      223189
2       217707
1233    214032
1236    213981
1239    213906
1234    213674
0       213121
671     210474
403     205024
46      197774
91      192532
609     181957
28      168914
188     130865
7       124711
49      121286
1235    119526
1238    119518
1237    119354
1240    119349
6       119066
39      118806
762     116046
65      114624
29      108476
84      107651
23      105095
180     104463
167     103887
596     101106
59       94998
517      94035
92       91807
67       90259
97       90164
603      89142
48       85754
80       82437
144      76450
75       74833
73       74411
156      72950
71       72922
36       71412
1027     69999
396      69320
40       67249
613      67132
205      65479
63       61666
89       61626
269      60686
1139     59841
433      59425
171      58901
981      58035
94       57373
673      57291
108      56916
34       55946
101      5

In [20]:
(data['banner_id'].value_counts()>100).sum()

1109

In [21]:
len(data['banner_id'].unique())

1633

In [22]:
def feature_engineering(data):
    
    #let's leave only known banners - that had >100 shows on the train set
    train_data = data.loc[(pd.to_datetime(data['date_time']).dt.month==9) & (pd.to_datetime(data['date_time']).dt.day!=1)]
    known_banner_ids = set(train_data['banner_id'].value_counts().loc[lambda x: x > 100].index)
    data.loc[~data.banner_id.isin(known_banner_ids), 'banner_id']=-1
    data = pd.get_dummies(data, columns = ['banner_id'], drop_first=True, prefix=['banner'])
    print("Banner columns generated.")
    
    data.loc[(data.zone_id>4) & (data.zone_id<10), "zone_id"]=-1
    data.loc[(data.zone_id>9) & (data.zone_id<20), "zone_id"]=-2
    data.loc[(data.zone_id>19) & (data.zone_id<50), "zone_id"]=-3
    data.loc[(data.zone_id>49), "zone_id"]=-4
    data = pd.get_dummies(data, columns = ['zone_id'], drop_first=True, prefix=['zone'])
    print("Zone columns generated.")
    
    #normalize campaign clicks
    data.loc[(data.campaign_clicks>50), 'campaign_clicks']=50
    data.loc[:,'campaign_clicks']=data['campaign_clicks']/50

    
    data.loc[data.os_id>6, "os_id"] = 7
    data = pd.get_dummies(data, columns = ['os_id'], drop_first=True, prefix=['os'])
    print("OS columns generated.")
    
    data = pd.get_dummies(data, columns = ['country_id'], drop_first=True, prefix=['country'])
    print("Country columns generated.")
    
#     let's do interactions between country and banner id
#     even banner dummies worked for a long time so I decided not to do that
#     ban_cols = (ban_col for ban_col in data.columns if 'banner_' in ban_col)
#     country_cols = (country_col for country_col in data.columns if 'country_' in country_col)
#     for ban_col in ban_cols:
#         for country_col in country_cols:
#             data[ban_col+country_col]=data[ban_col]*data[country_col]
    
    data = data.drop(columns=['date', 'impressions', 'hour'])
    
    train_data = data.loc[(pd.to_datetime(data['date_time']).dt.month==9) & (pd.to_datetime(data['date_time']).dt.day!=1)]
    val_data = data.loc[(pd.to_datetime(data['date_time']).dt.month==10) & (pd.to_datetime(data['date_time']).dt.day==1)]
    test_data = data.loc[(pd.to_datetime(data['date_time']).dt.month==10) & (pd.to_datetime(data['date_time']).dt.day==2)]
    train_data = train_data.drop(columns=['date_time'])  
    val_data = val_data.drop(columns=['date_time']) 
    test_data = test_data.drop(columns=['date_time']) 
    
    
    return train_data, val_data, test_data


In [30]:
#let's create dataframes first, otherwise it's too long time
def cv(train_data, val_data, test_data, alphas_list=[0.2, 0.5, 1.0, 2, 5]):
    
    
    #make X and y
    train_columns = [column for column in train_data.columns if column!='clicks']
    train_X = train_data[train_columns]
    val_X = val_data[train_columns]
    test_X = test_data[train_columns]
    train_y = train_data['clicks']
    val_y = val_data['clicks']
    test_y = test_data['clicks']
    
    
    models_list=[]
    val_loss_list=[]
    best_model=None
    best_loss=100000
    best_alpha=-1
    
    #kinda grid search for the best regression coefficient
    for alpha in alphas_list:
        #as it was discussed during the lecture, the most correct from bayesian point of view is a model with L2 regularization
        curr_model = Ridge(alpha=alpha)
        curr_model.fit(train_X, train_y)
        val_y =curr_model.predict(val_X)
        logloss = log_loss(val_data['clicks'], val_y)
        models_list.append(curr_model)
        val_loss_list.append(logloss)
        
        print(f"For alpha {alpha} log loss on validation is {logloss}")
        if logloss<best_loss:
            best_loss=logloss
            best_alpha=alpha
            best_model=curr_model
            
    #print data for the best model     
    test_y = best_model.predict(test_X)
    test_logloss = log_loss(test_data['clicks'], test_y)
    print(f"Best validation log loss is {best_loss} for alpha {best_alpha}. Log loss on test data for this model is {test_logloss}")
    return best_model, models_list, val_loss_list
            
        
        
        
        
        

In [25]:
%%time
train_data, val_data, test_data = feature_engineering(data)

Banner columns generated.
Zone columns generated.
OS columns generated.
Country columns generated.
CPU times: user 8min 41s, sys: 5min 24s, total: 14min 5s
Wall time: 15min 12s


In [26]:
train_data.head()

,campaign_clicks,clicks,banner_0,banner_1,banner_2,banner_3,banner_4,banner_5,banner_6,banner_7,banner_8,banner_9,banner_10,banner_11,banner_12,banner_13,banner_14,banner_15,banner_16,banner_17,banner_18,banner_19,banner_20,banner_21,banner_22,banner_23,banner_24,banner_25,banner_26,banner_27,banner_28,banner_29,banner_30,banner_31,banner_33,banner_34,banner_35,banner_36,banner_37,banner_38,banner_39,banner_40,banner_41,banner_42,banner_43,banner_44,banner_45,banner_46,banner_47,banner_48,banner_49,banner_50,banner_51,banner_52,banner_53,banner_54,banner_55,banner_56,banner_57,banner_58,banner_59,banner_60,banner_61,banner_62,banner_63,banner_64,banner_65,banner_66,banner_67,banner_68,banner_69,banner_70,banner_71,banner_72,banner_73,banner_74,banner_75,banner_77,banner_78,banner_79,banner_80,banner_81,banner_82,banner_83,banner_84,banner_85,banner_86,banner_87,banner_88,banner_89,banner_90,banner_91,banner_92,banner_93,banner_94,banner_95,banner_96,banner_97,banner_98,banner_99,banner_100,banner_101,banner_102,banner_103,banner_104,banner_105,banner_106,banner_107,banner_108,banner_109,banner_110,banner_111,banner_112,banner_113,banner_114,banner_115,banner_116,banner_117,banner_118,banner_119,banner_120,banner_121,banner_122,banner_123,banner_124,banner_125,banner_126,banner_127,banner_128,banner_129,banner_130,banner_131,banner_132,banner_133,banner_134,banner_135,banner_136,banner_137,banner_138,banner_139,banner_140,banner_141,banner_142,banner_143,banner_144,banner_145,banner_146,banner_147,banner_148,banner_149,banner_150,banner_151,banner_152,banner_153,banner_154,banner_155,banner_156,banner_157,banner_158,banner_159,banner_160,banner_161,banner_163,banner_164,banner_165,banner_166,banner_167,banner_168,banner_169,banner_170,banner_171,banner_172,banner_173,banner_174,banner_175,banner_177,banner_178,banner_179,banner_180,banner_181,banner_182,banner_183,banner_184,banner_185,banner_186,banner_187,banner_188,banner_189,banner_190,banner_191,banner_192,banner_193,banner_194,banner_195,banner_196,banner_197,banner_198,banner_199,banner_201,banner_202,banner_203,banner_204,banner_205,banner_206,banner_208,banner_209,banner_211,banner_212,banner_213,banner_214,banner_215,banner_216,banner_217,banner_218,banner_219,banner_220,banner_221,banner_222,banner_223,banner_224,banner_225,banner_226,banner_227,banner_228,banner_230,banner_231,banner_232,banner_233,banner_234,banner_235,banner_237,banner_238,banner_239,banner_240,banner_241,banner_242,banner_243,banner_244,banner_245,banner_246,banner_248,banner_249,banner_250,banner_251,banner_253,banner_254,banner_255,banner_256,banner_258,banner_259,banner_260,banner_261,banner_262,banner_263,banner_264,banner_265,banner_266,banner_267,banner_268,banner_269,banner_270,banner_271,banner_272,banner_273,banner_274,banner_275,banner_276,banner_277,banner_278,banner_280,banner_281,banner_282,banner_283,banner_284,banner_285,banner_286,banner_287,banner_288,banner_289,banner_290,banner_291,banner_292,banner_293,banner_294,banner_295,banner_296,banner_297,banner_298,banner_299,banner_301,banner_302,banner_303,banner_305,banner_306,banner_307,banner_309,banner_310,banner_311,banner_312,banner_313,banner_314,banner_315,banner_316,banner_319,banner_320,banner_321,banner_322,banner_323,banner_324,banner_325,banner_326,banner_328,banner_330,banner_331,banner_332,banner_333,banner_334,banner_335,banner_336,banner_337,banner_338,banner_339,banner_340,banner_341,banner_342,banner_343,banner_344,banner_345,banner_347,banner_348,banner_349,banner_350,banner_351,banner_352,banner_353,banner_354,banner_355,banner_357,banner_358,banner_359,banner_360,banner_361,banner_362,banner_363,banner_364,banner_365,banner_366,banner_367,banner_368,banner_369,banner_370,banner_371,banner_372,banner_373,banner_374,banner_376,banner_377,banner_378,banner_380,banner_381,banner_382,banner_383,banner_384,banner_385,banner_386,banner_387,banner_388,banner_389,banner_390,banner_391,banner_393,banner_394,banner_396,ban

In [31]:
%%time

best_model, models_list, val_lost_list = cv(train_data, val_data, test_data)

For alpha 0.2 log loss on validation is 0.1614665911992156
For alpha 0.5 log loss on validation is 0.16146284866104554
For alpha 1.0 log loss on validation is 0.16145731979447353
For alpha 2 log loss on validation is 0.16144841059759107
For alpha 5 log loss on validation is 0.161432420678378
Best validation log loss is 0.161432420678378 for alpha 5. Log loss on test data for this model is 0.1434375425967045
CPU times: user 53min 7s, sys: 1h 13min 40s, total: 2h 6min 48s
Wall time: 1h 21min 32s


Итоговый log loss получился 0.1434375425967045